In [38]:
# James Maguire, Josh Peck
# ESE429 Final Presentation
import numpy as np

In [39]:
class qubit:
    angle = 0 #degrees
    def __init__(self, angle):
        self.angle = angle
    def __repr__(self):
        return f"qubit(∠{self.angle}°)"

    def setAngle(self, angle): #unused
        self.angle = angle

    def measure(self, basisAngle):
        # Measure self.angle in the basis of basisAngle
        #     Both angles in degrees
        angleRad = np.deg2rad(self.angle)
        basisRad = np.deg2rad(basisAngle)

        probability = np.sin(angleRad-basisRad)**2
        isOne = probability > np.random.uniform()
        self.angle = basisAngle + 90*isOne

        return self.angle


In [40]:
class myHistogram:
    hist = dict()
    def __init__(self):
        self.hist = dict()
    def add(self, entry):
        if entry not in self.hist:
            self.hist[entry] = 1
        else:
            self.hist[entry] += 1
    def get(self):
        return self.hist

In [54]:
#Test code for qubit measurement logic
N=100000
for sendAngle in (0, 45):
    for recvAngle in (0, 45):
        angle1 = myHistogram()
        for i in range(0,N):
            qubit0 = qubit(sendAngle) # 0 degree qubit
            angle1.add(qubit0.measure(recvAngle))
        print(f"Sent: {sendAngle}, Recieved:{recvAngle}")
        print(angle1.get())
        print("")

Sent: 0, Recieved:0
{0: 100000}

Sent: 0, Recieved:45
{135: 50096, 45: 49904}

Sent: 45, Recieved:0
{90: 49911, 0: 50089}

Sent: 45, Recieved:45
{45: 100000}



In [42]:
def B92(N=100,eve=False,debugPrint=False,checkSize=8):
    # B92 Simulation (w/ eve)
    # N = 100000
    # eve = False
    # debugPrint = False

    print("B92 Simulation:")
    print(f"      (N={N}, Eve={eve}, check={checkSize})")

    # Step 1: Alice and Bob pick their N-bit binary sequences
    AliceBasisString = list(np.random.randint(0,2,N))
    BobBasisString = list(np.random.randint(0,2,N))
    if debugPrint:
        print("Alice's Secret String:")
        print(AliceBasisString)
        print("\nBob's Basis String:")
        print(BobBasisString)

    # Step 2: Alice chooses qubits, sends them to Bob
    #    0: 0d, 1:45d
    Qubits = [qubit(45*b) for b in AliceBasisString]
    if debugPrint:
        print("\nAlice's Sent Angles\n",[45*b for b in AliceBasisString])

    if eve:
        # Step 2b: eve measures the sent qubits
        EveBasisString = list(np.random.randint(0,2,N))
        eveAngles = [q.measure(45*b) for (q,b) in zip(Qubits, EveBasisString)]
        if debugPrint:
            print("\nEve's Measured Angles\n", eveAngles)

    # Step 3: Bob measures qubits based on his basis
    #    0: 0d, 1:45d
    BobAngles = [q.measure(45*b) for (q,b) in zip(Qubits, BobBasisString)]
    if debugPrint:
        print("\nBob's Measured Angles\n", BobAngles)

    # Step 4: Bob identifies which angles are 'weird'
    oddAngles = [(1 if(a in (90,135)) else 0) for a in BobAngles]
    if debugPrint:
        print("\nOdd Angles:\n")
        print(oddAngles)

    # Step 5: Bob flips his 'guesses' on these bits, alice selects those bits from her sequence
    bobKey = list()
    for n in range(0,N):
        if oddAngles[n] == 1:
            bobKey.append(int(not(BobBasisString[n])))

    aliceKey = list()
    for n in range(0,N):
        if oddAngles[n] == 1:
            aliceKey.append(AliceBasisString[n])
    if debugPrint:
        print("\nAlice's Key")
        print(aliceKey)
        print("\nBob's Key")
        print(bobKey)


    #Step 6: checking
    bobCheck = bobKey[0:checkSize]
    aliceCheck = aliceKey[0:checkSize]
    if debugPrint:
        print("\nAlice Check Sequence: ",aliceCheck)
        print("Bob Check Sequence: ",bobCheck)

    if bobCheck != aliceCheck:
        print("ERROR DETECTED: GO AWAY EVE!")
        nErrs = sum([a!=b for (a,b) in zip(aliceKey,bobKey)])
        print(f"total error rate: {nErrs}/{len(aliceKey)}~={round(nErrs/len(aliceKey),5)}")
    else:
        print("Eve isn't around!")
        #Final: the keys are done!
        if debugPrint:
            print("\n Alice's Final Key\n",aliceKey[checkSize:])
            print("\n Bob's Final Key\n",bobKey[checkSize:])

        print(f"Key length (pre-check): {len(aliceKey)}/{N}~={round(len(aliceKey)/N,6)}")
    print("\n")

In [43]:
def BB84(N=100,eve=False,debugPrint=False,checkSize=8):
    # BB84 (w/ eve)
    # N = 100000
    # eve = False
    # debugPrint = False
    print("BB84 Simulation:")
    print(f"      (N={N}, Eve={eve}, check={checkSize})")


    # Step 1: Alice and Bob pick their N-bit binary sequences
    AliceBasisString = list(np.random.randint(0,2,N))
    AliceSecretString = list(np.random.randint(0,2,N))
    BobBasisString = list(np.random.randint(0,2,N))
    if debugPrint:
        print("Alice's Basis String:")
        print(AliceBasisString)
        print("\nAlice's Secret String")
        print(AliceSecretString)
        print("\nBob's Basis String:")
        print(BobBasisString)

    # Step 2: Alice chooses qubits, sends them to Bob
    AliceAngles = [((45 + 90*s) if b else (90*s)) 
                for (b,s) in zip(AliceBasisString, AliceSecretString)]
    if debugPrint:
        print("\nAlice's Sent Angles\n",AliceAngles)

    Qubits = [qubit(a) for a in AliceAngles]

    if eve:
        # Step 2b: eve measures the sent qubits
        EveBasisString = list(np.random.randint(0,2,N))
        eveAngles = [q.measure(45*b) for (q,b) in zip(Qubits, EveBasisString)]
        if debugPrint:
            print("\nEve's Measured Angles\n", eveAngles)

    # Step 3: Bob measures qubits based on his basis
    #    0: 0d, 1:45d
    BobAngles = [q.measure(45*b) for (q,b) in zip(Qubits, BobBasisString)]
    if debugPrint:
        print("\nBob's Measured Angles\n", angles)

    # Step 4: Bob and alice compare measurement bases, find what bits they agreed on
    sameBasis = [int(a==b) for (a,b) in zip(AliceBasisString, BobBasisString)]
    if debugPrint:
        print("\nSameBasis:\n")
        print(sameBasis)

    # Step 5: Bob flips his 'guesses' on these bits
    #         Alice selects those bits from her sequence

    bobKey = list()
    for n in range(0,N):
        if sameBasis[n] == 1:
            bobKey.append(1 if BobAngles[n] in (90,135) else 0)


    aliceKey = list()
    for n in range(0,N):
        if sameBasis[n] == 1:
            aliceKey.append(AliceSecretString[n])

    if debugPrint:
        print("\nAlice's Key")
        print(aliceKey)
        print("\nBob's Key")
        print(bobKey)


    #Step 6: checking

    bobCheck = bobKey[0:checkSize]
    aliceCheck = aliceKey[0:checkSize]

    if debugPrint:
        print("\nAlice Check Sequence: ",aliceCheck)
        print("Bob Check Sequence: ",bobCheck)

    if bobCheck != aliceCheck:
        print("ERROR DETECTED: GO AWAY EVE!")
        nErrs = sum([a!=b for (a,b) in zip(aliceKey,bobKey)])
        print(f"total error rate: {nErrs}/{len(aliceKey)}~={round(nErrs/len(aliceKey),5)}")
    else:
        print("Eve isn't around!")
        #Final: the keys are done!
        if debugPrint:
            print("\n Alice's Final Key\n",aliceKey[checkSize:])
            print("\n Bob's Final Key\n",bobKey[checkSize:])
        print(f"Key length (pre-check): {len(aliceKey)}/{N}~={round(len(aliceKey)/N,6)}")
    print("\n")

In [44]:
B92(N=10**6,eve=False,debugPrint=False,checkSize=32)
B92(N=10**6,eve=True,debugPrint=False,checkSize=32)
BB84(N=10**6,eve=False,debugPrint=False,checkSize=32)
BB84(N=10**6,eve=True,debugPrint=False,checkSize=32)

B92 Simulation:
      (N=1000000, Eve=False, check=32)
Eve isn't around!
Key length (pre-check): 249923/1000000~=0.249923


B92 Simulation:
      (N=1000000, Eve=True, check=32)
ERROR DETECTED: GO AWAY EVE!
total error rate: 125063/375416~=0.33313


BB84 Simulation:
      (N=1000000, Eve=False, check=32)
Eve isn't around!
Key length (pre-check): 500093/1000000~=0.500093


BB84 Simulation:
      (N=1000000, Eve=True, check=32)
ERROR DETECTED: GO AWAY EVE!
total error rate: 124757/499122~=0.24995




In [50]:
B92(N=100,eve=True,debugPrint=True,checkSize=8)

B92 Simulation:
      (N=100, Eve=True, check=8)
Alice's Secret String:
[1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0]

Bob's Basis String:
[0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0]

Alice's Sent Angles
 [45, 45, 0, 0, 0, 45, 0, 45, 45, 45, 0, 0, 0, 45, 0, 45, 0, 0, 0, 45, 0, 45, 45, 45, 45, 45, 45, 0, 0, 45, 45, 45, 45, 45, 45, 45, 45, 45, 0, 0, 45, 45, 45, 45, 45, 45, 0, 0, 45, 0, 45, 0, 45, 45, 0, 0, 0, 0, 45, 0, 0, 0, 0, 0, 0, 45, 0, 45, 0, 45, 45, 45, 0, 0, 45, 0, 45, 0, 0, 45,